In [1]:
#LOAD DATA

import pickle

x_train = pickle.load(open('X_train_pre.pickle', 'rb'))
x_test = pickle.load(open('X_test_pre.pickle', 'rb'))
y_train = pickle.load(open('y_train.pickle', 'rb'))
y_test = pickle.load(open('y_test.pickle', 'rb'))

In [2]:
def add_counts(df):
    tmp_card = df[["C1", "C2", "C3", "C4", "C5"]]
    tmp_suit = df[["S1", "S2", "S3", "S4", "S5"]]
    #Pairs, sets, and quads. (equal ranks)
    df["cnt_c1"] = tmp_card.apply(lambda c: sum(c==c[0]), axis=1)
    df["cnt_c2"] = tmp_card.apply(lambda c: sum(c==c[1]), axis=1)
    df["cnt_c3"] = tmp_card.apply(lambda c: sum(c==c[2]), axis=1)
    df["cnt_c4"] = tmp_card.apply(lambda c: sum(c==c[3]), axis=1)
    df["cnt_c5"] = tmp_card.apply(lambda c: sum(c==c[4]), axis=1)
    # Flushes (five cards with the same suit)
    df["cnt_s1"] = tmp_suit.apply(lambda s: sum(s==s[0]), axis=1)
    df["cnt_s2"] = tmp_suit.apply(lambda s: sum(s==s[1]), axis=1)
    df["cnt_s3"] = tmp_suit.apply(lambda s: sum(s==s[2]), axis=1)
    df["cnt_s4"] = tmp_suit.apply(lambda s: sum(s==s[3]), axis=1)
    df["cnt_s5"] = tmp_suit.apply(lambda s: sum(s==s[4]), axis=1)
    return df

def add_unique_count(df):
    tmp_suit = df[["S1", "S2", "S3", "S4", "S5"]]
    df["unique_suit"] = tmp_suit.apply(lambda s: len(np.unique(s)), axis=1)
    return df

def add_diffs(df):
    tmp = df
    #if a straight is possible
    df["diff_1"] = tmp["C2"] - tmp["C1"]
    df["diff_2"] = tmp["C3"] - tmp["C2"]
    df["diff_3"] = tmp["C4"] - tmp["C3"]
    df["diff_4"] = tmp["C5"] - tmp["C4"]
    #Counts how many similar differences there are
    tmp_diff = df[["diff_1","diff_2","diff_3","diff_4"]]
    df["cnt_diff1"] = tmp_diff.apply(lambda c: sum(c==c[0]), axis=1)
    df["cnt_diff2"] = tmp_diff.apply(lambda c: sum(c==c[1]), axis=1)
    df["cnt_diff3"] = tmp_diff.apply(lambda c: sum(c==c[2]), axis=1)
    df["cnt_diff4"] = tmp_diff.apply(lambda c: sum(c==c[3]), axis=1)
    return df

def preprocess_data(data):
    df = data.copy()
    cards = df[["C1", "C2", "C3", "C4", "C5"]]
    suits = df[["S1", "S2", "S3", "S4", "S5"]]
    cards.values.sort()
    suits.values.sort()
    df[["C1", "C2", "C3", "C4", "C5"]] = cards
    df[["S1", "S2", "S3", "S4", "S5"]] = suits
    df = df[["S1", "C1","S2", "C2","S3", "C3","S4", "C4","S5", "C5"]]
    df = add_counts(df)
    df = add_diffs(df)
    df = add_unique_count(df)
    return df

In [3]:
import sys
!{sys.executable} -m pip install catboost

In [4]:
#CATBOOST
import numpy as np
from catboost import CatBoostClassifier
from catboost import Pool

pool_train = Pool(data = x_train, label = y_train)
pool_test = Pool(data = x_test, label = y_test)

model = CatBoostClassifier(iterations = 100)
model.fit(pool_train, verbose = 10)

Learning rate set to 0.5
0:	learn: 0.0908452	total: 225ms	remaining: 22.3s
10:	learn: 0.0039765	total: 565ms	remaining: 4.57s
20:	learn: 0.0018555	total: 867ms	remaining: 3.26s
30:	learn: 0.0011064	total: 1.17s	remaining: 2.61s
40:	learn: 0.0007489	total: 1.49s	remaining: 2.14s
50:	learn: 0.0005444	total: 1.78s	remaining: 1.71s
60:	learn: 0.0005251	total: 2.07s	remaining: 1.32s
70:	learn: 0.0004129	total: 2.37s	remaining: 967ms
80:	learn: 0.0004009	total: 2.65s	remaining: 622ms
90:	learn: 0.0002799	total: 2.95s	remaining: 292ms
99:	learn: 0.0002746	total: 3.22s	remaining: 0us


In [5]:
a = model.predict(data = x_test)

In [6]:
score = []
for i in range(10):
    score.append(0)
for i in a:
    score[int(i)] +=1
print(score)

[501208, 422498, 47622, 21132, 3885, 1996, 1424, 219, 12, 3]


In [10]:
import pandas as pd

card = np.array([[1, 1, 1, 10, 1, 11, 1, 12, 1, 13]])
card_df = pd.DataFrame(card, columns = ["S1", "C1", "S2", "C2", "S3", "C3", "S4", "C4", "S5", "C5"])
card_df = preprocess_data(card_df)
pred = model.predict(data = card_df)
print(pred[0][0])

9


In [8]:
x_test

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,...,cnt_s5,diff_1,diff_2,diff_3,diff_4,cnt_diff1,cnt_diff2,cnt_diff3,cnt_diff4,unique_suit
0,2,2,3,5,3,5,3,11,4,12,...,1,3,0,6,1,1,1,1,1,3
1,1,2,1,4,3,6,3,9,4,9,...,1,2,2,3,0,2,2,1,1,3
2,1,1,2,4,2,6,3,13,3,13,...,2,3,2,7,0,1,1,1,1,3
3,1,2,2,7,2,9,3,10,4,11,...,1,5,2,1,1,1,1,2,2,4
4,1,3,1,4,3,5,4,6,4,12,...,2,1,1,1,6,3,3,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,1,1,2,6,2,9,3,9,4,12,...,1,5,3,0,3,1,2,1,2,4
999995,1,3,2,3,3,4,4,5,4,7,...,2,0,1,1,2,1,2,2,1,4
999996,1,7,1,7,2,9,3,11,4,13,...,1,0,2,2,2,1,3,3,3,4
999997,1,1,1,8,2,8,3,11,3,13,...,2,7,0,3,2,1,1,1,1,3


In [9]:
pickle.dump(model, open('cat_boost.pkl', 'wb'))